<a href="https://colab.research.google.com/github/Astridgiraldo/RH/blob/main/Limpieza_y_transformacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Una empresa que tiene alrededor de 4000 empleados tiene una tasa de retiros de alrededor del 15% anual. Esto quiere decir que el 15% de los empleados en un año se retiran de la compañía por diferentes motivos. Los directivos de la empresa consideran que este nivel de retiros es muy alto, ya que trae altos costos para la empresa como: el costo de contratación que incluye todos los procesos desde el reclutamiento, la selección, la contratación y adicionalmente existe un costo relacionado con la capacitación, la curva de aprendizaje y las contrataciones no efectivas que son personas que se retiran de la empresa en el periodo de prueba. Adicionalmente, el retiro de las personas conlleva otras implicaciones negativas para la empresa que se presentan a continuación:

Los proyectos en los que los empleados trabajan se atrasan y se pueden ver comprometidas las fechas establecidas y esto puede llegar a afectar satisfacción de clientes y usuarios.
El trabajo del área de selección se aumenta y por lo tanto implica tener un área de mayor tamaño para el reclutamiento de las personas que se van.
Los empleados que permanecen tienen que trabajar más para cubrir las labores de las personas que se retiran y para la capacitación de las nuevas que llegan.
El conocimiento que tenían las personas y su experiencia se pierde y, debido a esto, el tiempo para ejecutar procesos aumenta.

Objetivos: Identificar los empleados que posiblemente vayan a renunciar para tomar medidas y en cuanto al modelo, reducir los falsos negativos (personas que realmente van a renunciar pero el modelo identifica que no lo van a hacer)

Modelos: Regresión logística sin selectores (alto recall)

decision tree classifier con tuneo de hiperparámetros (bajo recall y aumento de las demás métricas)

Limpieza: Eliminación de columnas (variables) con baja correlación y que tengan un mismo valor que no explique la variable objetivo según criterio cualitativo y cuantitativo.

Separar las BD de acuerdo a las fechas.

In [1]:
#Carga de librerías

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import sqlite3 as sql
from google.colab import drive
from sklearn.impute import SimpleImputer

In [2]:
#Conexión a Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Carga de BD desde drive

data_employee='/content/drive/MyDrive/analitica3/base_datos/employee_survey_data .csv'
genera_data= '/content/drive/MyDrive/analitica3/base_datos/general_data .csv'
data_manager='/content/drive/MyDrive/analitica3/base_datos/manager_survey.csv'
retirement = '/content/drive/MyDrive/analitica3/base_datos/retirement_info .csv'


df_employees=pd.read_csv(data_employee)
df_general=pd.read_csv(genera_data)
df_manager=pd.read_csv(data_manager)
df_retirement=pd.read_csv(retirement)

pd.set_option('display.max_columns', None) #muestra todas las columnas de tablas
%matplotlib inline

In [4]:
# Se hace una observación de la BD df_employees
df_employees.head(5)

,Unnamed: 0,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,DateSurvey
0,0,1,3.0,4.0,2.0,2015-12-31
1,1,2,3.0,2.0,4.0,2015-12-31
2,2,3,2.0,2.0,1.0,2015-12-31
3,3,4,4.0,4.0,3.0,2015-12-31
4,4,5,4.0,1.0,3.0,2015-12-31


In [5]:
# Se hace una observación de la BD df_general
df_general.head(5)

,Unnamed: 0,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,InfoDate
0,0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,Healthcare Representative,Married,131160,1.0,Y,11,8,0,1.0,6,1,0,0,2015-12-31
1,1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,Research Scientist,Single,41890,0.0,Y,23,8,1,6.0,3,5,1,4,2015-12-31
2,2,32,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,Sales Executive,Married,193280,1.0,Y,15,8,3,5.0,2,5,0,3,2015-12-31
3,3,38,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,3,Human Resources,Married,83210,3.0,Y,11,8,3,13.0,5,8,7,5,2015-12-31
4,4,32,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,1,Sales Executive,Single,23420,4.0,Y,12,8,2,9.0,2,6,0,4,2015-12-31


In [6]:
# Se hace una observación de la BD df_manager
df_manager.head(5)

,Unnamed: 0,EmployeeID,JobInvolvement,PerformanceRating,SurveyDate
0,0,1,3,3,2015-12-31
1,1,2,2,4,2015-12-31
2,2,3,3,3,2015-12-31
3,3,4,2,3,2015-12-31
4,4,5,3,3,2015-12-31


In [7]:
# convertir todos los caracteres de las bases de datos en minúscula.

def conver_min(df):
    df.columns = map(str.lower,df.columns)
conver_min(df_employees)
conver_min(df_general)
conver_min(df_manager)
conver_min(df_retirement)

# Tratamientos de nulos.


In [8]:
df_general.isnull().sum() #Consulta de cantidad de datos nulos en df_general

unnamed: 0                  0
age                         0
businesstravel              0
department                  0
distancefromhome            0
education                   0
educationfield              0
employeecount               0
employeeid                  0
gender                      0
joblevel                    0
jobrole                     0
maritalstatus               0
monthlyincome               0
numcompaniesworked         38
over18                      0
percentsalaryhike           0
standardhours               0
stockoptionlevel            0
totalworkingyears          18
trainingtimeslastyear       0
yearsatcompany              0
yearssincelastpromotion     0
yearswithcurrmanager        0
infodate                    0
dtype: int64

In [9]:
df_employees.isnull().sum() #Consulta de cantidad de datos nulos en df_employees

unnamed: 0                  0
employeeid                  0
environmentsatisfaction    50
jobsatisfaction            40
worklifebalance            76
datesurvey                  0
dtype: int64

In [10]:
df_manager.isnull().sum() #Consulta de cantidad de datos nulos en df_manger

unnamed: 0           0
employeeid           0
jobinvolvement       0
performancerating    0
surveydate           0
dtype: int64

In [11]:
df_retirement.isnull().sum() #Consulta de cantidad de datos nulos en df_retirement

unnamed: 0.1          0
unnamed: 0            0
employeeid            0
attrition             0
retirementdate        0
retirementtype        0
resignationreason    70
dtype: int64

En base de df_general se obtuvieron 38 nulos para la columna numcompaniesworked y 18 totalworkingyears.

En la base df_employee se obtuvieron 50 nulos para la columna environmentsatisfaction y 40 para jobsatisfaction 76 para worklifebalance.

En df_manager no hubo datos faltantes.

En df_retirement se obtuvieron 70 datos nulos para resignationreason.

Para un tratamiento de datos donde el numero de valores nulos no es significativo se utilizo la tecnica de imputación la cual es una técnica para estimar valores faltantes basado en datos observados.

In [12]:
# Imputación rellenando los nulos con el dato más frecuente

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df_general[['numcompaniesworked','totalworkingyears']]=imp.fit_transform(df_general[['numcompaniesworked','totalworkingyears']])
df_employees[['environmentsatisfaction', 'jobsatisfaction', 'worklifebalance']]=imp.fit_transform(df_employees[['environmentsatisfaction', 'jobsatisfaction', 'worklifebalance']])
df_retirement[['resignationreason']]=imp.fit_transform(df_retirement[['resignationreason']])

Se verifica los tamaños de las bases de datos.


In [13]:
print(df_general.shape)
print(df_employees.shape)
print(df_manager.shape)
print(df_retirement.shape)

(8820, 25)
(8820, 6)
(8820, 5)
(711, 7)


Revision tipologia de datos

In [14]:
df_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   unnamed: 0               8820 non-null   int64  
 1   age                      8820 non-null   int64  
 2   businesstravel           8820 non-null   object 
 3   department               8820 non-null   object 
 4   distancefromhome         8820 non-null   int64  
 5   education                8820 non-null   int64  
 6   educationfield           8820 non-null   object 
 7   employeecount            8820 non-null   int64  
 8   employeeid               8820 non-null   int64  
 9   gender                   8820 non-null   object 
 10  joblevel                 8820 non-null   int64  
 11  jobrole                  8820 non-null   object 
 12  maritalstatus            8820 non-null   object 
 13  monthlyincome            8820 non-null   int64  
 14  numcompaniesworked      

In [15]:
df_employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   unnamed: 0               8820 non-null   int64  
 1   employeeid               8820 non-null   int64  
 2   environmentsatisfaction  8820 non-null   float64
 3   jobsatisfaction          8820 non-null   float64
 4   worklifebalance          8820 non-null   float64
 5   datesurvey               8820 non-null   object 
dtypes: float64(3), int64(2), object(1)
memory usage: 413.6+ KB


In [16]:
df_manager.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   unnamed: 0         8820 non-null   int64 
 1   employeeid         8820 non-null   int64 
 2   jobinvolvement     8820 non-null   int64 
 3   performancerating  8820 non-null   int64 
 4   surveydate         8820 non-null   object
dtypes: int64(4), object(1)
memory usage: 344.7+ KB


In [17]:
df_retirement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   unnamed: 0.1       711 non-null    int64 
 1   unnamed: 0         711 non-null    int64 
 2   employeeid         711 non-null    int64 
 3   attrition          711 non-null    object
 4   retirementdate     711 non-null    object
 5   retirementtype     711 non-null    object
 6   resignationreason  711 non-null    object
dtypes: int64(3), object(4)
memory usage: 39.0+ KB


Analisis Descriptivo / manejo de datos son sql

In [18]:
#Conexión a Drive para guardar y cargar las BD con sentencias de SQL

conn= sql.connect('/content/drive/MyDrive/analitica3/base_datos/base_db')
cur=conn.cursor()

In [19]:
#Guardado de las BD en la carpeta para SQL

df_employees.to_sql('data_employee',conn,if_exists='replace')
df_general.to_sql('general_data',conn,if_exists='replace')
df_manager.to_sql('data_manager',conn,if_exists='replace')
df_retirement.to_sql('retirement',conn,if_exists='replace')

711

In [20]:
# consultar bases de datos que se pueden consultar por medio de sql

cur.execute("Select name from sqlite_master where type='table'")
cur.fetchall()

[('t1',),
 ('t2',),
 ('t3',),
 ('t4',),
 ('employee_2015',),
 ('employee_2016',),
 ('genera_data',),
 ('general_data2015',),
 ('general_data2016',),
 ('data_manager2015',),
 ('data_manager2016',),
 ('data_retirement2015',),
 ('data_retirement2016',),
 ('data_employee',),
 ('general_data',),
 ('data_manager',),
 ('retirement',)]

**Separación de las BD por año (2015 y 2016)**